In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

PROBLEM 1: Data analysis using markov chians 

In this problem, you will empirically analyze a Markov chain 
with a finite state space. Transition probabilities are unknown.

The state space is:
    S = {0, 1, 2, 3}

You are given the data for the observed X_t for t  = 0..19

Tasks:
1. Estimate the transition matrix P from the observed transitions.
2. Verify that the estimated matrix is a probability transition matrix.
3. Compute the stationary distribution pi of the chain.
4. Simulate the chain using the estimated transition matrix
5. Compute the expected hitting times via

   (a) Simulation

   (b) Solving linear equations (analytical hitting times). 

Compare the estimates and interpret the results


In [2]:
import numpy as np

# state space
S = [0, 1, 2, 3]
N_states = len(S)

# Observed transitions: each row is (current_state, next_state)
X_t = np.array([
    [0, 1],
    [1, 2],
    [2, 3],
    [3, 0],
    [0, 1],
    [1, 1],
    [1, 2],
    [2, 2],
    [2, 3],
    [3, 3],
    [3, 0],
    [0, 2],
    [2, 1],
    [1, 3],
    [3, 1],
    [1, 0],
    [0, 0],
    [0, 1],
    [1, 2],
    [2, 0],
], dtype=int)




Below are methods that you need to complete

In [ ]:
import numpy as np

# ============================================================
# 1.1 Estimate transition matrix from observed transitions
# ============================================================

def comp_transition_matrix(transitions, n_states):
    """
    Estimate the transition matrix P from observed transitions.

    transitions : array of shape (n_samples, 2)
        Each row contains (current_state, next_state)
    n_states : int
        Total number of states in the Markov chain

    Returns:
        P_hat : (n_states x n_states) numpy array
            Empirical estimate of transition probabilities
    """

    # Initialize matrix of transition counts
    P_hat = np.zeros((n_states, n_states), dtype=float)

    # Count how many times each transition i -> j occurs
    for i, j in transitions:
        P_hat[i, j] += 1.0

    # Compute total outgoing transitions from each state i
    row_sums = P_hat.sum(axis=1, keepdims=True)

    # Normalize rows to convert counts into probabilities
    # States with no outgoing transitions remain zero rows
    with np.errstate(divide='ignore', invalid='ignore'):
        P_hat = np.where(row_sums > 0, P_hat / row_sums, 0.0)

    return P_hat


# ============================================================
# 1.2 Check if a matrix is a valid transition matrix
# ============================================================

def is_transition_matrix(P, tol=1e-12):
    """
    Check whether P is a valid Markov transition matrix.

    Conditions:
    1. P is square
    2. All entries are non-negative
    3. Each row sums to 1 (within numerical tolerance)
    """

    P = np.asarray(P, dtype=float)

    # Check matrix is square
    if P.ndim != 2 or P.shape[0] != P.shape[1]:
        return False

    # Check non-negativity (allow tiny numerical errors)
    if np.any(P < -tol):
        return False

    # Each row must sum to 1
    row_sums = P.sum(axis=1)
    return np.all(np.abs(row_sums - 1.0) <= 1e-8)


# ============================================================
# 1.3 Compute stationary distribution
# ============================================================

def stationary_distribution(P):
    """
    Compute a stationary distribution pi such that:

        pi P = pi
        sum(pi) = 1
        pi_i >= 0

    Uses the eigenvector method with a linear-system fallback.
    """

    P = np.asarray(P, dtype=float)

    # Compute eigenvalues and eigenvectors of P^T
    # Stationary distribution is a left eigenvector with eigenvalue 1
    w, v = np.linalg.eig(P.T)

    # Find eigenvalue closest to 1
    idx = np.argmin(np.abs(w - 1.0))
    pi = np.real(v[:, idx])

    # Enforce non-negativity (eigenvectors are defined up to sign)
    pi = np.where(pi < 0, -pi, pi)

    # Normalize to make it a probability distribution
    s = pi.sum()

    if s == 0:
        # Fallback: solve (P^T - I) pi = 0 with sum(pi) = 1
        n = P.shape[0]
        A = np.vstack([P.T - np.eye(n), np.ones(n)])
        b = np.zeros(n + 1)
        b[-1] = 1

        pi, *_ = np.linalg.lstsq(A, b, rcond=None)
        pi = np.maximum(pi, 0)
        pi = pi / pi.sum()
    else:
        pi = pi / s

    return pi


# ============================================================
# 1.4 Simulate a Markov chain trajectory
# ============================================================

def simulate_chain(P, start_state, n_steps):
    """
    Simulate a Markov chain path with a fixed random seed.

    Returns:
        path : array of length n_steps + 1
            path[t] is the state at time t
    """

    seed = 1234  # fixed seed for reproducibility
    rng = np.random.default_rng(seed)

    P = np.asarray(P, dtype=float)
    n_states = P.shape[0]

    path = np.zeros(n_steps + 1, dtype=int)
    path[0] = start_state

    cur = start_state
    for t in range(1, n_steps + 1):
        # Sample next state using the transition probabilities
        cur = rng.choice(n_states, p=P[cur])
        path[t] = cur

    return path


# ============================================================
# 1.5 Monte Carlo estimation of hitting times
# ============================================================

def hitting_times_sim(P, start_state, n_sim=10_000):
    """
    Estimate expected hitting times E[T_{start -> j}] by simulation.

    Convention:
    - T_{j -> j} is the first return time (t >= 1)

    Returns:
        est[j] = estimated expected number of steps to hit state j
    """

    P = np.asarray(P, dtype=float)
    n_states = P.shape[0]

    seed = 1234
    rng = np.random.default_rng(seed)

    est = np.full(n_states, np.nan, dtype=float)

    for target in range(n_states):
        times = np.empty(n_sim, dtype=float)

        for s in range(n_sim):
            cur = start_state
            steps = 0

            # Simulate until target is reached
            while True:
                cur = rng.choice(n_states, p=P[cur])
                steps += 1
                if cur == target:
                    times[s] = steps
                    break

        est[target] = times.mean()

    return est


# ============================================================
# 1.6 Analytical (theoretical) hitting times
# ============================================================

def theoretical_hitting_times(P, start_state):
    """
    Compute expected hitting times analytically using linear equations.

    For each target j:
        m_i = E_i[T_j]
        m_i = 1 + sum_{k != j} P[i,k] m_k

    Returns:
        hit_theor[j] = E_{start_state}[T_j]
    """

    P = np.asarray(P, dtype=float)
    n_states = P.shape[0]

    hit_theor = np.full(n_states, np.nan, dtype=float)

    for target in range(n_states):

        # Initialize system A m = b
        A = np.eye(n_states)
        b = np.ones(n_states)

        # Build equations using first-step analysis
        for i in range(n_states):
            for k in range(n_states):
                if k == target:
                    continue
                A[i, k] -= P[i, k]

        # Solve linear system
        m = np.linalg.solve(A, b)

        # Extract hitting time from start_state
        hit_theor[target] = m[start_state]

    return hit_theor


When you are done, run the following cell (no need to implement anything else)

In [4]:
def problem1_main():
    print("\n=== Problem 1: Markov chain estimation + hitting times ===")

    # 1) Estimate P
    P_hat = comp_transition_matrix(X_t, N_states)
    print("Estimated P_hat:\n", np.round(P_hat, 3))

    # 2) Validate
    print("Is valid transition matrix?", is_transition_matrix(P_hat))

    # 3) Expected steps from given start state to all states
    start_state = 0

    # simulation
    mc = hitting_times_sim(P_hat, start_state=start_state, n_sim=5000)

    # Theory (linear system)
    th = theoretical_hitting_times(P_hat, start_state=start_state)

    # 4) Compare
    df = pd.DataFrame({
        "target_state": np.arange(N_states),
        "MC_estimate": mc,
        "theoretical": th,
        "abs_diff": np.abs(mc - th),
    })
    print("\nComparison table:\n", df)

PROBLEM 2: Cost-Sensitive Classification

You are given a binary classification problem for fraud detection.

Class labels:

    y = 1 => fraud

    y = 0 => ok



The costs of classification outcomes are:
    TP = 0, TN = 0, FP = 100, FN = 500

Tasks:
1. Train an SVM classifier.
2. Compute classification costs at a fixed threshold (0.5).
3. Evaluate total cost for multiple probability thresholds.
4. Find the threshold that minimizes total cost.

In [5]:
import numpy as np
import pandas as pd

costs = {"TP": 0, "TN": 0, "FP": 100, "FN": 500}


def generate_fraud_table(seed=0, n=3000, fraud_rate=0.05):
    """
    Generate a simple fraud dataset as a single table. The table contains:
        - numerical features: x1, x2, x3
        - binary target column: fraud (1 = fraud, 0 = legitimate)
    """
    rng = np.random.default_rng(seed)

    # Target variable
    fraud = (rng.random(n) < fraud_rate).astype(int)

    # Features
    x1 = rng.normal(0, 1, size=n)
    x2 = rng.normal(0, 1, size=n)
    x3 = rng.normal(0, 1, size=n)

    #  fraud cases are shifted
    x1[fraud == 1] += 2.0
    x2[fraud == 1] += 1.0

    df = pd.DataFrame({
        "x1": x1,
        "x2": x2,
        "x3": x3,
        "fraud": fraud,
    })

    return df


fraud_data = generate_fraud_table()

fraud_data.head()

,x1,x2,x3,fraud
0,-0.250243,-0.863902,-0.307019,0
1,-0.380736,0.018756,-0.559577,0
2,1.126431,2.055912,0.973126,1
3,0.806991,2.104160,-0.211368,1
4,0.059649,0.652374,-0.437259,0


Fill in the methods in the cell below:

In [ ]:
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split


def train_test_split_table(df, test_size=0.3, seed=0):
    """
    Split a data table into training and test sets.

    Returns:
        X_train, X_test, y_train, y_test
    """
    # Features and target
    X = df[["x1", "x2", "x3"]]
    y = df["fraud"].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        random_state=seed,
        stratify=y
    )

    return X_train, X_test, y_train.to_numpy(), y_test.to_numpy()


def fit_linear_svm(fraud_data):
    """Fit a linear SVM classifier and return the trained model."""
    clf = LinearSVC(
        C=1.0,
        max_iter=10_000,
        random_state=0
    )

    X_train, X_test, y_train, y_test = train_test_split_table(fraud_data)
    clf.fit(X_train, y_train)

    return clf      # Here clf is returned and not y_pred becuase the function is used later on in a way that reuires the model, not its predictions. 
                    # See exam template tutor doc for more on this.



def confusion_counts(y_true, y_pred):
    """Computes TP, TN, FP, FN."""
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)

    TP_est = int(np.sum((y_true == 1) & (y_pred == 1)))
    TN_est = int(np.sum((y_true == 0) & (y_pred == 0)))
    FP_est = int(np.sum((y_true == 0) & (y_pred == 1)))
    FN_est = int(np.sum((y_true == 1) & (y_pred == 0)))

    return {"TP": TP_est, "TN": TN_est, "FP": FP_est, "FN": FN_est}



def total_cost(counts):
    """Compute total cost from confusion counts."""
    return (
        counts["TP"] * costs["TP"]
        + counts["TN"] * costs["TN"]
        + counts["FP"] * costs["FP"]
        + counts["FN"] * costs["FN"]
    )


# evaluate how the classification cost changes when you change the decision threshold.
def sweep_thresholds(y_true, clf, X, thresholds):
    """Evaluate total cost for a range of thresholds.

    Notes:
      - LinearSVC has no predict_proba by default; we use decision_function scores.
      - Thresholds are applied to the decision scores.
    """

    results = []

    y_scores = clf.decision_function(X)

    for t in thresholds:
        y_pred = (y_scores >= t).astype(int)
        counts = confusion_counts(y_true, y_pred)
        cost = total_cost(counts)

        results.append({
            "threshold": float(t),
            "TP": counts["TP"],
            "TN": counts["TN"],
            "FP": counts["FP"],
            "FN": counts["FN"],
            "total_cost": float(cost),
        })

    return pd.DataFrame(results)


When you are done, run the following cell (no need to implement anything else)

In [7]:
def main():

    df = fraud_data

    print("Dataset head:")
    print(df.head(), "\n")

    # split in train and test:
    _, X_test, _, y_test = train_test_split_table(df)

    # Fit linear SVM
    clf = fit_linear_svm(df)

    # thresholds
    thresholds = np.linspace(-2.0, 2.0, 21)
    df_results = sweep_thresholds(
        y_test,
        clf,
        X_test,
        thresholds,
    )

    print("Threshold sweep results:")
    print(df_results)

    # Identify optimal threshold
    best_row = df_results.loc[df_results["total_cost"].idxmin()]
    print("Optimal threshold row:\n", best_row)


PROBLEM 3: Confidence estimation of the cost

In Problem 2, you trained a classifier, selected a decision threshold, evaluated its performance on a test set, and computed the cost

In this problem, you will quantify the uncertainty of this estimated cost. Each observation in the test set produces a cost depending on the
classification outcome:

    TN: 0
   
    FP: 100

    TP: 0

    FN: 500

Thus, the cost per observation is a bounded random variable taking
values in the interval [0, 500].

Tasks:
1. Compute the average cost per observation on the test set.
2. Use Hoeffding’s inequality to construct a 95% confidence interval
   for the true expected cost of the classifier.
3. Interpret the resulting interval:
   - What does it say about the reliability of your estimate?
   - Is the interval likely to be tight or conservative? Why?

You may assume that test observations are independent and identically
distributed.

In [8]:
def per_observation_cost(y_true, y_pred):
    """Compute per-observation cost vector."""
    y_true = np.asarray(y_true).astype(int)
    y_pred = np.asarray(y_pred).astype(int)

    c = np.zeros_like(y_true, dtype=float)

    # FP: y=0, pred=1 -> 100
    c[(y_true == 0) & (y_pred == 1)] = costs["FP"]

    # FN: y=1, pred=0 -> 500
    c[(y_true == 1) & (y_pred == 0)] = costs["FN"]

    # TP/TN -> 0 already
    return c



def hoeffding_ci(per_obs_costs, mean=None, n=None, a=0.0, b=500.0, delta=0.05):
    """Hoeffding confidence interval for the expected cost.

    Args:
        per_obs_costs: array-like of per-observation costs
        mean: (optional) mean cost (ignored; recomputed)
        n: (optional) sample size (ignored; recomputed)
        a, b: bounds of the random variable (here 0 and 500)
        delta: significance level (0.05 -> 95% CI)

    Returns:
        (lower, upper)
    """
    c = np.asarray(per_obs_costs, dtype=float)
    n = int(c.size)
    mean_cost = float(np.mean(c))

    # Hoeffding: P(|mean - E| >= eps) <= 2 exp(-2 n eps^2 / (b-a)^2)
    eps = (b - a) * np.sqrt(np.log(2.0 / delta) / (2.0 * n))

    lower = max(a, mean_cost - eps)
    upper = min(b, mean_cost + eps)

    return (lower, upper)


In [9]:
# --- Run all problems ---

# Problem 1
problem1_main()

# Problem 2
main()

# Problem 3: compute Hoeffding CI for cost at optimal threshold found above

# Recreate test split + trained model
_, X_test, _, y_test = train_test_split_table(fraud_data)
clf = fit_linear_svm(fraud_data)

# Find optimal threshold by sweeping
thresholds = np.linspace(-2.0, 2.0, 21)
df_results = sweep_thresholds(y_test, clf, X_test, thresholds)
best_row = df_results.loc[df_results["total_cost"].idxmin()]
opt_t = float(best_row["threshold"])

# Predictions at optimal threshold
scores = clf.decision_function(X_test)
y_pred_opt = (scores >= opt_t).astype(int)

c = per_observation_cost(y_test, y_pred_opt)
mean_cost = float(np.mean(c))
ci = hoeffding_ci(c, a=0.0, b=500.0, delta=0.05)

print("\n=== Problem 3: Hoeffding CI for mean cost ===")
print("Optimal threshold:", opt_t)
print("Mean cost per observation:", mean_cost)
print("95% Hoeffding CI:", ci)



=== Problem 1: Markov chain estimation + hitting times ===
Estimated P_hat:
 [[0.2   0.6   0.2   0.   ]
 [0.167 0.167 0.5   0.167]
 [0.2   0.2   0.2   0.4  ]
 [0.5   0.25  0.    0.25 ]]
Is valid transition matrix? True

Comparison table:
    target_state  MC_estimate  theoretical  abs_diff
0             0       3.9982     4.000000  0.001800
1             1       2.0030     2.024390  0.021390
2             2       3.3206     3.317073  0.003527
3             3       5.6616     5.682927  0.021327
Dataset head:
         x1        x2        x3  fraud
0 -0.250243 -0.863902 -0.307019      0
1 -0.380736  0.018756 -0.559577      0
2  1.126431  2.055912  0.973126      1
3  0.806991  2.104160 -0.211368      1
4  0.059649  0.652374 -0.437259      0 

Threshold sweep results:
    threshold  TP   TN   FP  FN  total_cost
0        -2.0  48  233  619   0     61900.0
1        -1.8  48  337  515   0     51500.0
2        -1.6  48  428  424   0     42400.0
3        -1.4  48  533  319   0     31900.0
4    

/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b
/Users/bradleydeku/Library/Python/3.9/lib/python/site-